In [2]:
import paho.mqtt.client as PahoMQTT
import time
from raspClient import raspClient
from dataset import SignalGenerator
import tensorflow as tf
import numpy as np
import pathlib
import os
import datetime
from datetime import timezone
import json
from collections import Counter

In [3]:
# DATASET PREPARATION -----------------------------------------
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# download dataset
data_dir = pathlib.Path('data/mini_speech_commands')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

# lista di test
test_list=[]
file=open("./kws_test_split.txt")
for line in file:
  test_list.append('.'+line[1:-1])

# lista di labels - da replicare in EX1
labels = open('labels.txt').readlines()[0].split() 
print(labels)

MFCC_OPTIONS = {
    'frame_length': 640, 
    'frame_step': 320, 
    'mfcc': True,
    'lower_frequency': 20, 
    'upper_frequency': 4000, 
    'num_mel_bins': 40,
    'num_coefficients': 10
}

# make test dataset
generator = SignalGenerator(labels, 16000, **MFCC_OPTIONS)     
test_ds = generator.make_dataset(test_list, False)

# COOPERATIVE CLIENT (RASP)----------------------------------------------------------------
timestamp = int(datetime.datetime.now(timezone.utc).timestamp())
device_name = 'rasp'

subscribe_topics, publish_topic = ['/group11IOTHW3/model/1', 
                                   '/group11IOTHW3/model/2',
                                   '/group11IOTHW3/model/3',
                                   '/group11IOTHW3/model/4'], '/group11IOTHW3/prep_sample'

rasp = raspClient('raspClient', subscribe_topics, publish_topic, 4)
rasp.start()

for i, (mfcc, label) in enumerate(test_ds):
    senml_msg = {
        "bn" : device_name,
        "bt" : timestamp,
        "e" :[{"n":"audio", "u":"/", "t":0, "vd":mfcc.numpy().tolist(), 'idx': i}]
        }
    senml_msg = json.dumps(senml_msg)
    rasp.myPublish(publish_topic, senml_msg, i, label.numpy())
    
    time.sleep(0.1) # tune this parameter

['down', 'stop', 'right', 'left', 'up', 'yes', 'no', 'go']
Connected to mqtt.eclipseprojects.io with result code: 0
(3, [])
(1, [])
(7, [])
(6, [])
(5, [])
(1, [])
(4, [])
(0, [])
(3, [])
(0, [])
(5, [])
(1, [])
(7, [])
0
0
0
0
(7, [])
(3, [])
(1, [])
(1, [])
1
2
1
2
1
2
1
2
(3, [])
3
3
3
3
(2, [])
4
4
4
4
(6, [])
(3, [])
5
6
5
6
5
6
5
(7, [])
6
7
7
7
7
(1, [])
8
8
8
(7, [])
9
10
(5, [])
9
10
8
9
9
11
10
10
11
(1, [])
(6, [])
(5, [])
11
12
13
11
12
13
14
12
13
(3, [])
14
12
(6, [])
13
14
15
16
15
16
14
(2, [])
15
15
(0, [])
(3, [])
16
16
(7, [])
17
18
19
17
18
19
17
18
19
(0, [])
17
18
19
(2, [])
20
21
20
21
(6, [])
20
20
21
21
(2, [])
(2, [])
22
22
22
22
(3, [])
23
23
24
24
25
25
23
24
(6, [])
25
23
24
25
(6, [])
26
27
26
26
(0, [])
27
27
26
27
(4, [])
(5, [])
28
28
28
28
29
30
29
29
30
30
29
30
(6, [])
(7, [])
31
31
31
31
32
32
32
(3, [])
33
(7, [])
33
33
32
33
34
34
34
34
(1, [])
(3, [])
35
36
35
35
36
36
(0, [])
35
36
37
(6, [])
37
38
39
37
37
38
(1, [])
(6, [])
38
39
39
38
39
40
4

(0, [])
348
348
346
346
(7, [])
347
348
347
348
(3, [])
(1, [])
349
349
350
350
349
349
350
(4, [])
351
351
351
352
350
(5, [])
352
352
351
352
(5, [])
353
353
353
354
353
354
354
354
355
355
(6, [])
(6, [])
355
355
356
356
357
358
359
(6, [])
356
357
358
(5, [])
359
356
357
358
359
360
361
357
358
359
(4, [])
(1, [])
360
361
360
361
362
363
364
(1, [])
(0, [])
360
361
362
363
364
362
363
364
365
(7, [])
366
362
363
364
365
366
(7, [])
367
365
366
367
368
(1, [])
(4, [])
369
370
365
366
367
368
(5, [])
368
369
370
(2, [])
371
367
368
369
370
371
(6, [])
369
370
371
372
373
(4, [])
(7, [])
372
373
374
375
372
373
(6, [])
374
371
372
(7, [])
373
374
374
375
375
376
(6, [])
377
378
376
376
377
(3, [])
(6, [])
378
375
376
377
378
379
(6, [])
379
377
378
(5, [])
379
380
381
380
381
380
(3, [])
379
380
382
381
382
(1, [])
381
382
382
383
383
384(3, [])

(6, [])
384
383
384
(3, [])
385
385
386
386
383
384
(4, [])
385
386
385
387
387
(4, [])
388
388
389
386
387
389
(5, [])
390
387
388
(2, [])


(0, [])
692
693
694
(7, [])
694
694
695
696
695
696
(3, [])
(1, [])
695
696
695
696
697
(7, [])
697
697
697
698
698
699
(6, [])
700
698
699
(1, [])
700
698
699
699
700
700
(7, [])
(0, [])
701
701
702
702
701
(4, [])
701
702
702
703
703
703
(1, [])
703
704
704
(7, [])
704
705
705
705
704
705
(7, [])
706
706
706
706
707
(2, [])
707
707
(5, [])
707
708
708
708
(6, [])
708
709
709
(6, [])
709
709
710
710
710
710
(0, [])
711
711
711
711
712
(3, [])
712
712
712
713
713
713
(7, [])
713
(4, [])
714
714
(3, [])
714
714
715
715
715
715
(4, [])
716
716
717
717
716
(4, [])
717
716
717
718
718
718
(3, [])
718
719
719
(4, [])
(2, [])
719
719
720
720
720
720
(7, [])
721
721
721
721
722
(4, [])
722
722
723
723
723
722
(6, [])
723
724
724
(6, [])
724
724
725
725
(3, [])
725
725
(5, [])
726
726
726
726
727
(1, [])
727
727
727
728
728
728
(4, [])
729
729
729
(5, [])
(7, [])
728
729
730
730
730
730
(2, [])
731
731
731
731
732
(0, [])
732
732
732
(4, [])
733
733
734
734
733
733
(6, [])
734
734
735(0, [])



In [ ]:
from collections import Counter
import numpy as np
a = [1,2,3,4,3,4,4,5,5,1,2,2,2]
counter = Counter(a)
elements = list(counter.items())
elements.sort(key= lambda x:x[1], reverse=True)
elements[0][0]

In [ ]:
a = dict()
a['a']